In [1]:
!wget https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar -O tabular-dl-tabr.tar.gz
!tar -xvf tabular-dl-tabr.tar.gz

--2024-11-16 11:57:58--  https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar
Resolving huggingface.co (huggingface.co)... 3.171.171.104, 3.171.171.128, 3.171.171.65, ...
Connecting to huggingface.co (huggingface.co)|3.171.171.104|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/1b/25/1b25d6e2556c827abfaf6ba9da6021338691cafc39fe9d77986955ae0a69243d/0d74e4b96febb48fbe4dd2d851f8638b5738354f82c3183f92adbd2abf2f256b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data.tar%3B+filename%3D%22data.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1732017478&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjAxNzQ3OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xYi8yNS8xYjI1ZDZlMjU1NmM4MjdhYmZhZjZiYTlkYTYwMjEzMzg2OTFjYWZjMzlmZTlkNzc5ODY5NTVhZTBhNjkyNDNkLzBkNzRlNGI5NmZlYmI0OGZiZTRkZDJkODUxZjg2MzhiNTczODM1NGY4MmMzMTgzZjkyYWRiZDJhYmYyZjI1N

In [2]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.0 MB/s eta 0:00:00


In [3]:
!pip install psutil

In [4]:
import pandas as pd
import numpy as np
#import optuna
import torch.optim
import psutil
import sklearn.preprocessing

from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
clf = TabNetClassifier()

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [5]:
X_num_train = np.load('/content/data/classif-cat-medium-0-electricity/X_num_train.npy')
X_cat_train = np.load('/content/data/classif-cat-medium-0-electricity/X_cat_train.npy')
y_train = np.load('/content/data/classif-cat-medium-0-electricity/Y_train.npy')

X_num_valid = np.load('/content/data/classif-cat-medium-0-electricity/X_num_val.npy')
X_cat_valid = np.load('/content/data/classif-cat-medium-0-electricity/X_cat_val.npy')
y_valid = np.load('/content/data/classif-cat-medium-0-electricity/Y_val.npy')

X_num_test = np.load('/content/data/classif-cat-medium-0-electricity/X_num_test.npy')
X_cat_test = np.load('/content/data/classif-cat-medium-0-electricity/X_cat_test.npy')
y_test = np.load('/content/data/classif-cat-medium-0-electricity/Y_test.npy')

In [6]:
normalizer = sklearn.preprocessing.QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X_num_train.shape[0] // 30, 1000), 10),
            subsample=1_000_000_000,
            )
normalizer.fit_transform(X_num_train)

array([[-0.59145314, -1.0403167 ,  0.5123105 , ...,  0.27523908,
        -0.02265221, -0.4073718 ],
       [-0.12870593,  0.47394076, -0.44041222, ..., -0.4697212 ,
        -0.74904263,  0.60227716],
       [ 0.4697212 ,  0.7129065 ,  0.6839989 , ...,  1.2536699 ,
         1.5527538 ,  0.16262195],
       ...,
       [ 0.43725622, -0.08314662, -1.1479142 , ..., -1.1979828 ,
         1.086838  ,  0.8931596 ],
       [ 0.17432515,  0.2948974 , -1.2764194 , ..., -1.3734854 ,
        -2.5847876 ,  1.637109  ],
       [-1.2031567 ,  1.2031567 ,  0.6324262 , ...,  0.27523908,
        -0.02265221, -0.4073718 ]], dtype=float32)

In [7]:
encoder = sklearn.preprocessing.OrdinalEncoder(
        handle_unknown='use_encoded_value',  # type: ignore[code]
        unknown_value=np.iinfo('int64').max - 3,  # type: ignore[code]
        dtype='int64',  # type: ignore[code]
    )
encoder.fit_transform(X_cat_train)

array([[1],
       [6],
       [3],
       ...,
       [5],
       [0],
       [2]])

In [8]:
X_train = np.concatenate([X_num_train, X_cat_train], axis=1)
X_valid = np.concatenate([X_num_valid, X_cat_valid], axis=1)
X_test = np.concatenate([X_num_test, X_cat_test], axis=1)

In [ ]:
#Baseline
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
)

epoch 0  | loss: 0.75505 | train_accuracy: 0.5366  | valid_accuracy: 0.53524 |  0:00:02s
epoch 1  | loss: 0.6052  | train_accuracy: 0.5035  | valid_accuracy: 0.50293 |  0:00:03s
epoch 2  | loss: 0.54888 | train_accuracy: 0.5071  | valid_accuracy: 0.50878 |  0:00:04s
epoch 3  | loss: 0.52156 | train_accuracy: 0.5171  | valid_accuracy: 0.51908 |  0:00:05s
epoch 4  | loss: 0.50685 | train_accuracy: 0.5127  | valid_accuracy: 0.51264 |  0:00:05s
epoch 5  | loss: 0.50359 | train_accuracy: 0.499   | valid_accuracy: 0.49988 |  0:00:06s
epoch 6  | loss: 0.49805 | train_accuracy: 0.5064  | valid_accuracy: 0.50632 |  0:00:07s
epoch 7  | loss: 0.49591 | train_accuracy: 0.5016  | valid_accuracy: 0.50176 |  0:00:08s
epoch 8  | loss: 0.49504 | train_accuracy: 0.5065  | valid_accuracy: 0.50585 |  0:00:09s
epoch 9  | loss: 0.49848 | train_accuracy: 0.5168  | valid_accuracy: 0.51627 |  0:00:09s
epoch 10 | loss: 0.48804 | train_accuracy: 0.5205  | valid_accuracy: 0.52259 |  0:00:10s

Early stopping occur

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
def objective(trial):
    params = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        'momentum': trial.suggest_float('momentum', 0.01, 0.4),
    }

    model = TabNetClassifier(**params, n_a=params['n_d'])
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=['accuracy'], patience=10)

    # Evaluate model performance
    score = model.best_cost  # or any other metric

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params

[I 2024-07-06 07:03:52,962] A new study created in memory with name: no-name-f449d837-4c37-4268-9761-eeb2605ae7cc
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.15064 | val_0_accuracy: 0.52587 |  0:00:01s
epoch 1  | loss: 0.71539 | val_0_accuracy: 0.49801 |  0:00:03s
epoch 2  | loss: 0.6752  | val_0_accuracy: 0.52716 |  0:00:05s
epoch 3  | loss: 0.59623 | val_0_accuracy: 0.51498 |  0:00:06s
epoch 4  | loss: 0.60151 | val_0_accuracy: 0.49941 |  0:00:08s
epoch 5  | loss: 0.60043 | val_0_accuracy: 0.56392 |  0:00:10s
epoch 6  | loss: 0.58946 | val_0_accuracy: 0.53957 |  0:00:11s
epoch 7  | loss: 0.55048 | val_0_accuracy: 0.50105 |  0:00:13s
epoch 8  | loss: 0.53711 | val_0_accuracy: 0.53255 |  0:00:15s
epoch 9  | loss: 0.57197 | val_0_accuracy: 0.58113 |  0:00:16s
epoch 10 | loss: 0.5482  | val_0_accuracy: 0.5885  |  0:00:18s
epoch 11 | loss: 0.58315 | val_0_accuracy: 0.51651 |  0:00:20s
epoch 12 | loss: 0.54766 | val_0_accuracy: 0.6228  |  0:00:22s
epoch 13 | loss: 0.5245  | val_0_accuracy: 0.63358 |  0:00:23s
epoch 14 | loss: 0.51837 | val_0_accuracy: 0.63276 |  0:00:25s
epoch 15 | loss: 0.52624 | val_0_accuracy: 0.53571 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 07:04:34,521] Trial 0 finished with value: 0.6335752751112151 and parameters: {'n_d': 17, 'n_steps': 8, 'gamma': 1.9593723888140113, 'n_independent': 4, 'n_shared': 3, 'momentum': 0.06334497217538498}. Best is trial 0 with value: 0.6335752751112151.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.03294 | val_0_accuracy: 0.51475 |  0:00:01s
epoch 1  | loss: 0.64911 | val_0_accuracy: 0.50925 |  0:00:02s
epoch 2  | loss: 0.58597 | val_0_accuracy: 0.49965 |  0:00:04s
epoch 3  | loss: 0.57681 | val_0_accuracy: 0.50012 |  0:00:05s
epoch 4  | loss: 0.54917 | val_0_accuracy: 0.51709 |  0:00:06s
epoch 5  | loss: 0.53573 | val_0_accuracy: 0.51241 |  0:00:08s
epoch 6  | loss: 0.55246 | val_0_accuracy: 0.55186 |  0:00:09s
epoch 7  | loss: 0.54108 | val_0_accuracy: 0.54835 |  0:00:11s
epoch 8  | loss: 0.53643 | val_0_accuracy: 0.58745 |  0:00:12s
epoch 9  | loss: 0.5253  | val_0_accuracy: 0.53582 |  0:00:14s
epoch 10 | loss: 0.52706 | val_0_accuracy: 0.60454 |  0:00:15s
epoch 11 | loss: 0.517   | val_0_accuracy: 0.50574 |  0:00:16s
epoch 12 | loss: 0.51465 | val_0_accuracy: 0.51838 |  0:00:18s
epoch 13 | loss: 0.50205 | val_0_accuracy: 0.5048  |  0:00:19s
epoch 14 | loss: 0.49788 | val_0_accuracy: 0.53219 |  0:00:20s
epoch 15 | loss: 0.48936 | val_0_accuracy: 0.55397 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 07:05:23,153] Trial 1 finished with value: 0.6960899086864902 and parameters: {'n_d': 34, 'n_steps': 5, 'gamma': 1.9926804851631865, 'n_independent': 5, 'n_shared': 4, 'momentum': 0.10833978561532207}. Best is trial 1 with value: 0.6960899086864902.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 08:30:31,053] Trial 97 finished with value: 0.8011004448606883 and parameters: {'n_d': 24, 'n_steps': 3, 'gamma': 1.2799779891182257, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.3474104971381722}. Best is trial 97 with value: 0.8011004448606883.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.63214 | val_0_accuracy: 0.49813 |  0:00:00s
epoch 1  | loss: 0.52607 | val_0_accuracy: 0.54917 |  0:00:01s
epoch 2  | loss: 0.5101  | val_0_accuracy: 0.51604 |  0:00:01s
epoch 3  | loss: 0.49913 | val_0_accuracy: 0.51779 |  0:00:02s
epoch 4  | loss: 0.48822 | val_0_accuracy: 0.55526 |  0:00:02s
epoch 5  | loss: 0.4847  | val_0_accuracy: 0.53301 |  0:00:03s
epoch 6  | loss: 0.47755 | val_0_accuracy: 0.57153 |  0:00:03s
epoch 7  | loss: 0.47335 | val_0_accuracy: 0.50105 |  0:00:04s
epoch 8  | loss: 0.47182 | val_0_accuracy: 0.5007  |  0:00:04s
epoch 9  | loss: 0.46702 | val_0_accuracy: 0.60606 |  0:00:05s
epoch 10 | loss: 0.46908 | val_0_accuracy: 0.52798 |  0:00:06s
epoch 11 | loss: 0.46791 | val_0_accuracy: 0.63252 |  0:00:06s
epoch 12 | loss: 0.4708  | val_0_accuracy: 0.63428 |  0:00:07s
epoch 13 | loss: 0.46026 | val_0_accuracy: 0.62199 |  0:00:07s
epoch 14 | loss: 0.46383 | val_0_accuracy: 0.64868 |  0:00:08s
epoch 15 | loss: 0.46158 | val_0_accuracy: 0.64481 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 08:31:26,546] Trial 98 finished with value: 0.8026223366892999 and parameters: {'n_d': 25, 'n_steps': 3, 'gamma': 1.2930002861396965, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.32872996615004835}. Best is trial 98 with value: 0.8026223366892999.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.62802 | val_0_accuracy: 0.49953 |  0:00:00s
epoch 1  | loss: 0.52435 | val_0_accuracy: 0.49953 |  0:00:01s
epoch 2  | loss: 0.5068  | val_0_accuracy: 0.51346 |  0:00:01s
epoch 3  | loss: 0.49732 | val_0_accuracy: 0.50199 |  0:00:02s
epoch 4  | loss: 0.4882  | val_0_accuracy: 0.52494 |  0:00:02s
epoch 5  | loss: 0.4819  | val_0_accuracy: 0.51955 |  0:00:03s
epoch 6  | loss: 0.48502 | val_0_accuracy: 0.5247  |  0:00:03s
epoch 7  | loss: 0.48517 | val_0_accuracy: 0.51241 |  0:00:04s
epoch 8  | loss: 0.48267 | val_0_accuracy: 0.50176 |  0:00:04s
epoch 9  | loss: 0.47399 | val_0_accuracy: 0.53524 |  0:00:05s
epoch 10 | loss: 0.47641 | val_0_accuracy: 0.54062 |  0:00:05s
epoch 11 | loss: 0.47447 | val_0_accuracy: 0.54823 |  0:00:06s
epoch 12 | loss: 0.47202 | val_0_accuracy: 0.52283 |  0:00:07s
epoch 13 | loss: 0.46603 | val_0_accuracy: 0.51768 |  0:00:07s
epoch 14 | loss: 0.4673  | val_0_accuracy: 0.51651 |  0:00:08s
epoch 15 | loss: 0.46573 | val_0_accuracy: 0.53559 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 08:32:22,173] Trial 99 finished with value: 0.7970030437836573 and parameters: {'n_d': 25, 'n_steps': 3, 'gamma': 1.1875668635220615, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.3336552606437718}. Best is trial 98 with value: 0.8026223366892999.


In [ ]:
best_params

{'n_d': 25,
 'n_steps': 3,
 'gamma': 1.2930002861396965,
 'n_independent': 2,
 'n_shared': 2,
 'momentum': 0.32872996615004835}

In [9]:
best_params = {
    'n_d': 25,
    'n_steps': 3,
    'gamma': 1.2930002861396965,
    'n_independent': 2,
    'n_shared': 2,
    'momentum': 0.32872996615004835
    }

In [10]:
from pytorch_tabnet.callbacks import Callback

def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"System RAM Usage: {mem_info.rss / (1024 ** 2):.2f} MB")
    if torch.cuda.is_available():
      print(f"Allocated GPU memory: {torch.cuda.memory_allocated() / (1024 ** 2):.2f} MB")
      print(f"Cached GPU memory: {torch.cuda.memory_reserved() / (1024 ** 2):.2f} MB")

class MemoryUsageCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print_memory_usage()

In [ ]:
# Train final model using best hyperparameters
best_model = TabNetClassifier(
    **best_params,
    n_a=best_params['n_d'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2)
)

best_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    patience=16,
    max_epochs=200,
    callbacks=[MemoryUsageCallback()]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.63214 | train_accuracy: 0.4983  | valid_accuracy: 0.49813 |  0:00:05s
System RAM Usage: 1178.50 MB
Allocated GPU memory: 17.32 MB
Cached GPU memory: 44.00 MB
epoch 1  | loss: 0.52601 | train_accuracy: 0.5944  | valid_accuracy: 0.59237 |  0:00:07s
System RAM Usage: 1178.96 MB
Allocated GPU memory: 17.32 MB
Cached GPU memory: 44.00 MB
epoch 2  | loss: 0.50833 | train_accuracy: 0.4989  | valid_accuracy: 0.49953 |  0:00:09s
System RAM Usage: 1178.96 MB
Allocated GPU memory: 17.32 MB
Cached GPU memory: 44.00 MB
epoch 3  | loss: 0.49343 | train_accuracy: 0.5594  | valid_accuracy: 0.55373 |  0:00:10s
System RAM Usage: 1178.96 MB
Allocated GPU memory: 17.32 MB
Cached GPU memory: 44.00 MB
epoch 4  | loss: 0.49303 | train_accuracy: 0.5908  | valid_accuracy: 0.58979 |  0:00:11s
System RAM Usage: 1178.96 MB
Allocated GPU memory: 17.32 MB
Cached GPU memory: 44.00 MB
epoch 5  | loss: 0.48589 | train_accuracy: 0.5236  | valid_accuracy: 0.53009 |  0:00:12s
System RAM Usage: 1178.96 

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

Best Valid Score: 0.7954811519550457
Test AUC: 0.8811287320695776


In [ ]:
preds = best_model.predict(X_test)

print(classification_report(y_true=y_test, y_pred=preds))

              precision    recall  f1-score   support

           0       0.79      0.80      0.80      9951
           1       0.80      0.79      0.80      9981

    accuracy                           0.80     19932
   macro avg       0.80      0.80      0.80     19932
weighted avg       0.80      0.80      0.80     19932



In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_score=y_proba[:,1], y_true=y_test)

roc_data = pd.DataFrame({
    'fpr': fpr,
    'tpr': tpr
})

roc_data.to_csv('electricity-TabNet.csv', index=False)